In [ ]:
from moviepy.audio.io.AudioFileClip import AudioFileClip
import uuid
from pathlib import Path

In [ ]:
packages = """
accelerate==1.81,
moviepy==2.2.1,
torch==2.7.1,
torchaudio==2.7.1,
transformers==4.53.0,
pydantic==2.10.6,
uvicorn==0.34.0,
fastapi==0.115.6,
pydantic-settings==2.9.0,
fastapi==0.115.14,
pydantic==2.11.7,
uvicorn==0.35.0
"""


In [ ]:
!uv pip show torch torchaudio transformers accelerate moviepy pydantic uvicorn fastapi

In [ ]:
audio = AudioFileClip("test_audio.mp4")
out_file = f"test_audio-{uuid.uuid4()}.mp3"
audio.write_audiofile(out_file)
with open(out_file, "rb") as f:
    audio_bts = f.read()
Path(out_file).unlink()



In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True
)

In [ ]:
import io
import torchaudio

In [ ]:
# Function to process bytes
def bts_to_np(audio_bytes: bytes):
    # Convert bytes to waveform using torchaudio
    with io.BytesIO(audio_bytes) as audio_file:
        waveform, sample_rate = torchaudio.load(audio_file)

    # Whisper expects 16kHz audio
    if sample_rate != 16000:
        waveform = torchaudio.functional.resample(waveform, orig_freq=sample_rate, new_freq=16000)

    # Convert to mono if stereo
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # Convert waveform to numpy array
    waveform_np = waveform.squeeze().numpy()
    return waveform_np

In [ ]:
waveform_np = bts_to_np(audio_bts)

In [ ]:
result = pipe(waveform_np)

In [ ]:
transcription = result["text"]

In [ ]:
transcription

In [ ]:
result = pipe("test_audio.mp3")

In [ ]:
result

In [ ]:
x = result["text"]

In [ ]:
x

In [ ]:
transcription = result["text"].strip()

In [ ]:
transcription

In [ ]:


dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])
